# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("11-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'4610211 4 0 59 3907 201586 929 33750'

In [8]:
test_data_raw = r"""125 17"""

# test_data_raw2 = r"""0123
# 1234
# 8765
# 9876"""

def preprocess_data (data):
    # dtype='U10'
    
    # dtype='U10'
    rows = [int(stone) for stone in data.split(" ")]
    return rows



test_data = preprocess_data(test_data_raw)
# test_data2 = preprocess_data(test_data_raw2)
display(test_data)

[125, 17]

In [9]:
full_data = preprocess_data(data_raw)
full_data

[4610211, 4, 0, 59, 3907, 201586, 929, 33750]

In [12]:
def solution (data, verbose=False):
    

    max_blinks = 25

    def manual_step(stones, steps):
        current_stones = stones.copy()
        for step in range(steps):
            next_stones = []
            for stone in current_stones:
                if stone == 0:
                    next_stones.append(1)
                elif len(str(stone)) % 2 == 0:
                    stone_s = str(stone)
                    next_stones.append(int(stone_s[:len(stone_s)//2]))
                    next_stones.append(int(stone_s[len(stone_s)//2:]))
                else:
                    next_stones.append(stone*2024)
                current_stones = next_stones
        return next_stones

    lookup_dict = {i: [] for i in range(10)}
    for i in range(10):
        next_stones = [i]
        for step in range(max_blinks):
            next_stones = manual_step(next_stones, 1)
            lookup_dict[i].append(len(next_stones))

    # print(lookup_dict)


    def optim_step(stones, steps):
        current_stones = stones.copy()
        total_count = 0
        for step in range(steps):
            next_stones = []
            for stone in current_stones:
                if stone < 10:
                    steps_to_go = steps - step
                    total_count += lookup_dict[stone][steps_to_go-1]

                elif len(str(stone)) % 2 == 0:
                    stone_s = str(stone)
                    next_stones.append(int(stone_s[:len(stone_s)//2]))
                    next_stones.append(int(stone_s[len(stone_s)//2:]))
                else:
                    next_stones.append(stone*2024)
                current_stones = next_stones
        return total_count + len(current_stones)

    # print(optim_step(data, 6))
    # print(optim_step(data, 25))
        

    return optim_step(data, 25)



sol = solution(test_data, verbose=True)

print(sol)


55312


In [13]:
sol = solution(full_data)
print(sol)

197357


# Part 2

In [52]:
def solution2 (data, verbose=False):

    max_blinks = 75

    def next_stones (stone):
        if stone == 0:
            return [1]
        elif len(str(stone)) % 2 == 0:
            stone_s = str(stone)
            return [int(stone_s[:len(stone_s)//2]), int(stone_s[len(stone_s)//2:])]
        else:
            return [stone*2024]
    
    current_stones = data.copy()
    next_stones_memory = {}
    already_seen = data.copy()
    for step in tqdm(range(max_blinks)): 
        step_stones = []
        for stone in current_stones:
            if type(stone) == tuple:
                step_stones.append((stone[0], stone[1]+1))
            else:
                 
                if stone in next_stones_memory:
                    sub_stones = next_stones_memory[stone]
                else:
                    sub_stones = next_stones(stone)
                    next_stones_memory[stone] = sub_stones
                
                for sub_stone in sub_stones:
                    if sub_stone in already_seen:
                        step_stones.append((sub_stone, 1))
                    else:
                        already_seen.append(sub_stone)
                        step_stones.append(sub_stone)

        
        current_stones = step_stones
    
    memory = {(stone,0): 1  for stone in already_seen}
    for step in tqdm(range(1, max_blinks+1)): 
        for stone in already_seen:
            memory[(stone, step)] = sum([memory[(sub_stone, step-1)] for sub_stone in next_stones_memory[stone] ])
            
    return sum([memory[(stone, max_blinks)] for stone in data])




sol = solution2(test_data, verbose=True)
print(sol)
    

100%|██████████| 75/75 [00:00<00:00, 18745.77it/s]

65601038650482


In [53]:
sol = solution2(full_data)
print(sol)

100%|██████████| 75/75 [00:00<00:00, 369.46it/s]

234568186890978
